Step 1: Load and Merge Data

In [1]:
import pandas as pd
data = pd.read_csv('train_essays.csv')
development_data = pd.read_csv('development.csv')

Step 2: Build Vocabulary

In [2]:
import re
from collections import Counter

def buildVocabulary(texts, occurence=5):
    all_text = ' '.join(texts)
    words = re.findall(r'\b\w+\b', all_text.lower())
    word_counts = Counter(words)
    vocabulary = [word for word, count in word_counts.items() if count >= occurence]
    return vocabulary


In [3]:
vocabulary = buildVocabulary(data['text'])
print(vocabulary)

['cars', 'have', 'been', 'around', 'since', 'they', 'became', 'famous', 'in', 'the', '1900s', 'when', 'henry', 'ford', 'created', 'and', 'built', 'first', 'modelt', 'played', 'a', 'major', 'role', 'our', 'every', 'day', 'lives', 'then', 'but', 'now', 'people', 'are', 'starting', 'to', 'question', 'if', 'limiting', 'car', 'usage', 'would', 'be', 'good', 'thing', 'me', 'use', 'of', 'might', 'do', 'like', 'matter', 'this', 'article', 'german', 'suburb', 'life', 'goes', 'on', 'without', 'by', 'elizabeth', 'rosenthal', 'states', 'how', 'automobiles', 'linchpin', 'suburbs', 'where', 'middle', 'class', 'families', 'from', 'either', 'shanghai', 'or', 'chicago', 'tend', 'make', 'their', 'homes', 'experts', 'say', 'is', 'huge', 'impediment', 'current', 'efforts', 'reduce', 'greenhouse', 'gas', 'emissions', 'tailpipe', 'passenger', 'responsible', 'for', '12', 'percent', 'europe', 'up', '50', 'some', 'carintensive', 'areas', 'united', 'main', 'reason', 'because', 'lot', 'driving', 'them', 'all', '

Step 3: Create Reverse Index

In [4]:
def reverseIndex(vocabulary):
    reverse_index = {word: index for index, word in enumerate(vocabulary)}
    return reverse_index


In [5]:
print(reverseIndex(vocabulary))

{'cars': 0, 'have': 1, 'been': 2, 'around': 3, 'since': 4, 'they': 5, 'became': 6, 'famous': 7, 'in': 8, 'the': 9, '1900s': 10, 'when': 11, 'henry': 12, 'ford': 13, 'created': 14, 'and': 15, 'built': 16, 'first': 17, 'modelt': 18, 'played': 19, 'a': 20, 'major': 21, 'role': 22, 'our': 23, 'every': 24, 'day': 25, 'lives': 26, 'then': 27, 'but': 28, 'now': 29, 'people': 30, 'are': 31, 'starting': 32, 'to': 33, 'question': 34, 'if': 35, 'limiting': 36, 'car': 37, 'usage': 38, 'would': 39, 'be': 40, 'good': 41, 'thing': 42, 'me': 43, 'use': 44, 'of': 45, 'might': 46, 'do': 47, 'like': 48, 'matter': 49, 'this': 50, 'article': 51, 'german': 52, 'suburb': 53, 'life': 54, 'goes': 55, 'on': 56, 'without': 57, 'by': 58, 'elizabeth': 59, 'rosenthal': 60, 'states': 61, 'how': 62, 'automobiles': 63, 'linchpin': 64, 'suburbs': 65, 'where': 66, 'middle': 67, 'class': 68, 'families': 69, 'from': 70, 'either': 71, 'shanghai': 72, 'or': 73, 'chicago': 74, 'tend': 75, 'make': 76, 'their': 77, 'homes': 78

Step 4: Calculate Occurrence Probability

In [6]:
def occurrenceProbability(word, all_documents):
    total_words = sum(1 for doc in all_documents if word in doc)
    total_documents = len(all_documents)
    return total_words / total_documents


Step 5: Calculate Conditional Probability

In [7]:
def conditionalProbability(word, class_documents, all_documents):
    total_words = sum(1 for doc in class_documents if word in doc)
    total_documents = len(class_documents)
    return {word: total_words / total_documents} if total_documents > 0 else {}

Step 6: Calculate Conditional Probability with Laplace Smoothing

In [8]:
def conditionalProbabilitySmoothed(word, class_documents, all_documents, vocabulary_size, smoothing_parameter=1):
    total_words = sum(1 for doc in class_documents if word in doc)
    total_documents = len(class_documents)
    return (total_words + smoothing_parameter) / (total_documents + smoothing_parameter * vocabulary_size)


Step 7: Predict Class

In [9]:
import math
def predict(document, vocabulary, human_occurence, ai_occurence, human_conditional, ai_conditional):
    words = re.findall(r'\b\w+\b', document.lower())
    probability_human = math.log(human_occurence)
    probability_ai = math.log(ai_occurence)

    for word in words:
        if word in vocabulary:
            conditional_probs_word_human = human_conditional.get(word, 0)
            conditional_probs_word_llm = ai_conditional.get(word, 0)
            probability_human += math.log(conditional_probs_word_human)
            probability_ai += math.log(conditional_probs_word_llm)

    return "0" if probability_human > probability_ai else "1"


Step 8: Calculate Accuracy

In [10]:
def calculate_accuracy(dev_documents, dev_labels, vocabulary, human_occurence, ai_occurence, human_conditional, ai_conditional):
    correct_predictions = 0

    for doc, label in zip(dev_documents, dev_labels):
        predicted_class = predict(doc, vocabulary, human_occurence, ai_occurence, human_conditional, ai_conditional)
        if predicted_class == label:
            correct_predictions += 1

    accuracy = correct_predictions / len(dev_documents)
    return accuracy

In [11]:
human_train = data[data["generated"] == 0]["text"].tolist()
ai_train = data[data["generated"] == 1]["text"].tolist()

human_dev_essays = development_data[development_data['generated'] == 0]['text'].tolist()
ai_dev_essays = development_data[development_data['generated'] == 1]['text'].tolist()

dev_documents = human_dev_essays + ai_dev_essays
dev_labels = ["0"] * len(human_dev_essays) + ["1"] * len(ai_dev_essays)

vocabulary = buildVocabulary(human_train + ai_train)

human_occurence = occurrenceProbability("the", human_train)
ai_occurence = occurrenceProbability("the", ai_train)

vocabulary_size = len(vocabulary)

human_conditional = {word: conditionalProbability(word, human_train, human_train) for word in vocabulary}
ai_conditional = {word: conditionalProbability(word, ai_train, ai_train) for word in vocabulary}


In [12]:
print(human_conditional)
print(ai_conditional)

{'cars': {'cars': 0.403999375097641}, 'have': {'have': 0.8786127167630058}, 'been': {'been': 0.4260271832526168}, 'around': {'around': 0.31588814247773783}, 'since': {'since': 0.17450398375253867}, 'they': {'they': 0.7119200124980472}, 'became': {'became': 0.02124668020621778}, 'famous': {'famous': 0.013747851898140916}, 'in': {'in': 0.9987501952819872}, 'the': {'the': 0.9981252929229808}, '1900s': {'1900s': 0.0031245117950320263}, 'when': {'when': 0.45024214966411497}, 'henry': {'henry': 0.0}, 'ford': {'ford': 0.06155288236213092}, 'created': {'created': 0.06420871738790813}, 'and': {'and': 0.9943758787689424}, 'built': {'built': 0.03296359943758788}, 'first': {'first': 0.16059990626464615}, 'modelt': {'modelt': 0.0}, 'played': {'played': 0.05077331666927043}, 'a': {'a': 0.9990626464614903}, 'major': {'major': 0.14903921262302766}, 'role': {'role': 0.036400562412123104}, 'our': {'our': 0.8434619590688955}, 'every': {'every': 0.4280581159193876}, 'day': {'day': 0.5577253554132167}, 'li

Step 9: Compare the Effect of Smoothing

In [13]:
smoothing_parameters = [0.1, 1, 5, 10]

for smoothing_parameter in smoothing_parameters:
    conditional_probs_human_smoothed = {word: conditionalProbabilitySmoothed(word, human_train, human_train, vocabulary_size, smoothing_parameter) for word in vocabulary}
    conditional_probs_llm_smoothed = {word: conditionalProbabilitySmoothed(word, ai_train, ai_train, vocabulary_size, smoothing_parameter) for word in vocabulary}
    accuracy = calculate_accuracy(dev_documents, dev_labels, vocabulary, human_occurence, ai_occurence, conditional_probs_human_smoothed, conditional_probs_llm_smoothed)
    print(f"Smoothing Parameter: {smoothing_parameter}, Accuracy: {accuracy}")


Smoothing Parameter: 0.1, Accuracy: 0.8709981167608286
Smoothing Parameter: 1, Accuracy: 0.8662900188323918
Smoothing Parameter: 5, Accuracy: 0.852165725047081
Smoothing Parameter: 10, Accuracy: 0.8502824858757062


Step 10: Derive Top 10 Words Predicting Each Class

In [14]:
def topWords(class_probability, vocabulary, top_n=10):
    sorted_words = sorted(vocabulary, key=lambda word: class_probability[word], reverse=True)
    return sorted_words[:top_n]

In [ ]:
top_words_human = topWords(human_conditional, vocabulary)
top_words_ai = topWords(ai_conditional, vocabulary)

print("Top words for Human-generated essays:", top_words_human)
print("Top words for AI-generated essays:", top_words_ai)